In [5]:
import os
import pandas as pd
import csv

# Function to detect delimiter by reading the first line of the file
def detect_delimiter(txt_file_path):
    with open(txt_file_path, 'r') as file:
        first_line = file.readline()
        # Check common delimiters: comma, tab, and space
        if ',' in first_line:
            return ','
        elif '\t' in first_line:
            return '\t'
        elif ' ' in first_line:
            return ' '
        else:
            return None

# Function to convert a single .txt file to .xlsx, including header information
def convert_txt_to_xlsx(txt_file_path, output_folder):
    try:
        # Read the entire file to handle both header and data
        with open(txt_file_path, 'r') as file:
            # Assuming the header is the first few lines before the data table
            header_lines = []
            for i in range(5):  # Adjust the number of header lines accordingly
                line = file.readline().strip()
                header_lines.append(line)
        
        # Detect the delimiter
        delimiter = detect_delimiter(txt_file_path)
        if not delimiter:
            raise ValueError(f"Unknown delimiter in file: {txt_file_path}")

        # Read the actual data, skipping the header lines
        df = pd.read_csv(txt_file_path, delimiter=delimiter, skiprows=len(header_lines), engine="python", quoting=csv.QUOTE_NONE)
        
        # Prepare Excel writer
        base_name = os.path.basename(txt_file_path).replace('.txt', '.xlsx')  # Changed to .xlsx
        xlsx_file_path = os.path.join(output_folder, base_name)
        
        with pd.ExcelWriter(xlsx_file_path, engine='openpyxl') as writer:
            # Write header lines as text at the top of the Excel sheet
            header_df = pd.DataFrame(header_lines)
            header_df.to_excel(writer, sheet_name='Sheet1', index=False, header=False)

            # Append the main data underneath the header
            df.to_excel(writer, sheet_name='Sheet1', index=False, startrow=len(header_lines) + 2)
        
        print(f"Converted: {txt_file_path} to {xlsx_file_path}")

    except pd.errors.ParserError as e:
        print(f"ParserError while reading {txt_file_path}: {e}")
    except Exception as e:
        print(f"Error while converting {txt_file_path}: {e}")

# Main function to convert all .txt files in a folder to .xlsx
def batch_convert_txt_to_xlsx(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all the files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.txt'):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_txt_to_xlsx(txt_file_path, output_folder)

# Example usage:
input_folder = "/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-TXT"  # Replace with your folder path containing .txt files
output_folder = "/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX"  # Replace with the folder path where .xlsx files will be saved

batch_convert_txt_to_xlsx(input_folder, output_folder)


Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-TXT/AJR_12-4-23_VJ_T1 - NSUBBVertJump.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/AJR_12-4-23_VJ_T1 - NSUBBVertJump.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-TXT/GWL_12-4-23_VJ_T1 - NSUBBVertJump.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/GWL_12-4-23_VJ_T1 - NSUBBVertJump.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-TXT/KTC_12-6-23_VJ_T1 - NSUBBVertJump.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/KTC_12-6-23_VJ_T1 - NSUBBVertJump.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-TXT/NRF_12-4-23_VJ_T1 - NSUBBVertJump.txt to /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/NRF_12-

In [6]:
import os
import pandas as pd

# Define a function to calculate peak forces, percent differences, and the average of peak forces for two phases
def analyze_force_data(file_path):
    # Load the XLSX file, skipping the first 7 rows to start at the header (which is in row 8)
    df = pd.read_excel(file_path, skiprows=7)

    # Check if 'VEM1', 'VEM2', 'VEM3', and 'VEM4' exist in the columns
    required_columns = ['VEM1', 'VEM2', 'VEM3', 'VEM4', 'GRF-L', 'GRF-R']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Column {col} not found in {file_path}")

    # Find the event mark rows (VEM1, VEM2 for Phase 1, VEM3, VEM4 for Phase 2)
    if df[df['VEM1'] == 1].empty or df[df['VEM2'] == 1].empty:
        raise ValueError(f"Missing VEM1 or VEM2 markers in {file_path}")
    
    if df[df['VEM3'] == 1].empty or df[df['VEM4'] == 1].empty:
        raise ValueError(f"Missing VEM3 or VEM4 markers in {file_path}")
    
    start_idx_1 = df[df['VEM1'] == 1].index[0]  # Start of Phase 1
    end_idx_1 = df[df['VEM2'] == 1].index[0]    # End of Phase 1
    
    start_idx_2 = df[df['VEM3'] == 1].index[0]  # Start of Phase 2
    end_idx_2 = df[df['VEM4'] == 1].index[0]    # End of Phase 2

    # Extract data for each leg for both phases
    left_leg_force_1 = df['GRF-L'][start_idx_1:end_idx_1 + 1]  # 'GRF-L' is left leg force
    right_leg_force_1 = df['GRF-R'][start_idx_1:end_idx_1 + 1]  # 'GRF-R' is right leg force
    
    left_leg_force_2 = df['GRF-L'][start_idx_2:end_idx_2 + 1]
    right_leg_force_2 = df['GRF-R'][start_idx_2:end_idx_2 + 1]
    
    # Calculate peak forces for both phases
    peak_left_leg_force_1 = left_leg_force_1.max()
    peak_right_leg_force_1 = right_leg_force_1.max()
    
    peak_left_leg_force_2 = left_leg_force_2.max()
    peak_right_leg_force_2 = right_leg_force_2.max()
    
    # Calculate the average of the peak forces between VEM1 and VEM2
    avg_peak_force_1 = (peak_left_leg_force_1 + peak_right_leg_force_1) / 2
    
    # Calculate percent differences for both phases
    percent_difference_1 = abs(peak_left_leg_force_1 - peak_right_leg_force_1) / ((peak_left_leg_force_1 + peak_right_leg_force_1) / 2) * 100
    percent_difference_2 = abs(peak_left_leg_force_2 - peak_right_leg_force_2) / ((peak_left_leg_force_2 + peak_right_leg_force_2) / 2) * 100
    
    # Return the results for both phases, including average peak force
    return (avg_peak_force_1, peak_left_leg_force_1, peak_right_leg_force_1, percent_difference_1, 
            peak_left_leg_force_2, peak_right_leg_force_2, percent_difference_2)

# Set up the folder paths for input and output
input_folder_path = '/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX'
output_folder_path = '/Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-Result'

# Ensure the output folder exists
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

results = []

# Loop through all the files in the input folder
for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(input_folder_path, file_name)
        
        try:
            # Analyze the data from the file
            (avg_peak_force_1, peak_left_1, peak_right_1, percent_diff_1, 
             peak_left_2, peak_right_2, percent_diff_2) = analyze_force_data(file_path)
        
            # Extract participant initials from file name
            participant_initials = file_name[:3]
        
            # Append the results to the list
            results.append([participant_initials, avg_peak_force_1, peak_left_1, peak_right_1, percent_diff_1, 
                            peak_left_2, peak_right_2, percent_diff_2])
        
        except ValueError as e:
            print(f"Error processing file {file_name}: {e}")

# Convert the results into a DataFrame
results_df = pd.DataFrame(results, columns=[
    'Participant', 
    'Avg Peak Force Phase 1 (N)', 
    'Peak Left Leg Force Phase 1 (N)', 'Peak Right Leg Force Phase 1 (N)', 'Percent Difference Phase 1 (%)',
    'Peak Left Leg Force Phase 2 (N)', 'Peak Right Leg Force Phase 2 (N)', 'Percent Difference Phase 2 (%)'
])

# Define the output file path
output_file_path = os.path.join(output_folder_path, 'Dec2-6force_analysis_report.xlsx')

# Save the individual trial results to an Excel file with two sheets in the output folder
with pd.ExcelWriter(output_file_path) as writer:
    # Save individual trials results
    results_df.to_excel(writer, sheet_name='Individual Trials', index=False)
    
    # Group by participant and calculate average metrics
    grouped_results = results_df.groupby('Participant').mean().reset_index()
    
    # Save the averaged results to a separate sheet
    grouped_results.to_excel(writer, sheet_name='Averaged by Participant', index=False)

print(f"Combined report generated and saved to {output_file_path}")

Error processing file DCM_12-3-23_VJ_T1 - PowerVJ.xlsx: Column GRF-L not found in /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/DCM_12-3-23_VJ_T1 - PowerVJ.xlsx
Error processing file CMR_12-3-23_VJ_T1 - PowerVJ.xlsx: Column GRF-L not found in /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/CMR_12-3-23_VJ_T1 - PowerVJ.xlsx
Error processing file JTH_12-3-23_VJ_T1 - PowerVJ.xlsx: Column GRF-L not found in /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/JTH_12-3-23_VJ_T1 - PowerVJ.xlsx
Error processing file DCM_12-3-23_VJ_T2 - PowerVJ.xlsx: Column GRF-L not found in /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/DCM_12-3-23_VJ_T2 - PowerVJ.xlsx
Error processing file CMR_12-3-23_VJ_T2 - PowerVJ.xlsx: Column GRF-L not found in /Users/brenthokeness/Desktop/BiomechData/Fall2024Baseball-VJTesting/BBVJDec2-6-XLSX/CMR_12-3-23_VJ_T2 - PowerVJ.xlsx
Error